In [10]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd


# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns


# Estandarización variables numéricas y Codificación variables categóricas
# ------------------------------------------------------------------------------
from sklearn.preprocessing import StandardScaler

# Gestión datos desbalanceados
# ------------------------------------------------------------------------------
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTETomek

# Para separar los datos en train y test
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split

#  Gestión de warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

<font color= green>Usando el mismo dataset que usatéis ayer, los objetivos de los ejercicios de hoy son:
- Estandarizar las variables numéricas de vuestro set de datos
- Codificar las variables categóricas. Recordad que tendréis que tener en cuenta si vuestras variables tienen orden o no.
- Chequear si vuestros datos están balanceados. En caso de que no lo estén utilizad algunas de las herramientas aprendidas en la lección para balancearlos.
- Guardad el dataframe con los cambios que habéis aplicado para utilizarlo en la siguiente lección.

In [12]:
df = pd.read_pickle('Datos/Travel_Insurance_limpio.pkl')
df.head(2)

,Age,Employment_Type,Graduate_Or_Not,Annual_Income,Family_Members,Chronic_Diseases,Frequent_Flyer,Ever_Travelled_Abroad,Travel_Insurance
0,31,Government Sector,Yes,400000,6,1,No,No,0
1,31,Private Sector/Self Employed,Yes,1250000,7,0,No,No,0


In [14]:
df.dtypes

Age                         int64
Employment_Type          category
Graduate_Or_Not          category
Annual_Income               int64
Family_Members              int64
Chronic_Diseases            int64
Frequent_Flyer           category
Ever_Travelled_Abroad    category
Travel_Insurance            int64
dtype: object

### <font color=violet>Estandarización

Como ya sabemos que el método Standard Scaler es el más ampliamente usado, lo vamos a apicar directamente. 

In [15]:
# iniciamos el método para escalar

scaler = StandardScaler()

In [16]:
# recordemos que la estandarización solo se hace para las variables predictoras numéricas

numericas = df.select_dtypes(include = np.number)
numericas.head()

,Age,Annual_Income,Family_Members,Chronic_Diseases,Travel_Insurance
0,31,400000,6,1,0
1,31,1250000,7,0,0
2,34,500000,4,1,1
3,28,700000,3,1,0
4,28,700000,8,1,0


In [17]:
# Ajustamos nuestros datos
scaler.fit(numericas)

# transformamos los datos

X_escaladas = scaler.transform(numericas)

# por último convertiremos el array que nos devuelve en un dataframe. 

numericas_estandar = pd.DataFrame(X_escaladas, columns = numericas.columns)
numericas_estandar.head(2)

,Age,Annual_Income,Family_Members,Chronic_Diseases,Travel_Insurance
0,0.426113,-1.482446,0.629929,1.415063,-0.794071
1,0.426113,0.874835,1.197592,-0.706682,-0.794071


In [18]:
# Dropeamos las columnas sin estandarizar del df original.

df.drop(["Age", "Annual_Income", "Family_Members", "Chronic_Diseases", "Travel_Insurance"], axis = 1, inplace=True)
df.head()

,Employment_Type,Graduate_Or_Not,Frequent_Flyer,Ever_Travelled_Abroad
0,Government Sector,Yes,No,No
1,Private Sector/Self Employed,Yes,No,No
2,Private Sector/Self Employed,Yes,No,No
3,Private Sector/Self Employed,Yes,No,No
4,Private Sector/Self Employed,Yes,Yes,No


In [19]:
# Unimos las numéricas estándar al df original.

df = pd.concat([df, numericas_estandar], axis = 1)

# chequeamos que esta todo bien
df.head()

,Employment_Type,Graduate_Or_Not,Frequent_Flyer,Ever_Travelled_Abroad,Age,Annual_Income,Family_Members,Chronic_Diseases,Travel_Insurance
0,Government Sector,Yes,No,No,0.426113,-1.482446,0.629929,1.415063,-0.794071
1,Private Sector/Self Employed,Yes,No,No,0.426113,0.874835,1.197592,-0.706682,-0.794071
2,Private Sector/Self Employed,Yes,No,No,1.453557,-1.205119,-0.505398,1.415063,1.259334
3,Private Sector/Self Employed,Yes,No,No,-0.601330,-0.650464,-1.073061,1.415063,-0.794071
4,Private Sector/Self Employed,Yes,Yes,No,-0.601330,-0.650464,1.765256,1.415063,-0.794071


### <font color=violet>Codificación

In [20]:
categoricas = df.select_dtypes(include = 'category')
categoricas.head()

,Employment_Type,Graduate_Or_Not,Frequent_Flyer,Ever_Travelled_Abroad
0,Government Sector,Yes,No,No
1,Private Sector/Self Employed,Yes,No,No
2,Private Sector/Self Employed,Yes,No,No
3,Private Sector/Self Employed,Yes,No,No
4,Private Sector/Self Employed,Yes,Yes,No


Cre